# Dataset cleaning

### Importing data

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r'../scrap/mubawab_listings.csv', dtype=object)

### Delete unnecessary columns

In [2]:
df = df.drop(['Standing', 'Ascenseur', 'Vue_sur_mer', 'Garage'], axis=1)

### Property_State colum, if it's equal to 0, it's NAN

In [3]:
df['Property_State'] = df['Property_State'].replace('0', pd.NA)

### Convert the relevant columns to numeric

In [4]:
# Convert numerical columns to appropriate data types
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Area'] = pd.to_numeric(df['Area'], errors='coerce')
df['Rooms'] = pd.to_numeric(df['Rooms'], errors='coerce')
df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce')
df['Bathrooms'] = pd.to_numeric(df['Bathrooms'], errors='coerce')
df['Jardin'] = df['Jardin'].astype(int)
df['Piscine'] = df['Piscine'].astype(int)
df['Cuisine_equiped'] = df['Cuisine_equiped'].astype(int)

### delete rows missing rooms and status

In [5]:
# delete rows missing rooms and status
df = df.dropna(subset=['Rooms', 'Status'])

### Function to fill missing values in rooms and status based on similar properties

In [6]:
# Fill missing Price values step by step
df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Quartier', 'Status', 'Bedrooms', 'Bathrooms', 'Rooms', 'Property_State', 'Jardin', 'Piscine', 'Cuisine_equiped'])['Price']
    .transform(lambda x: x.mean()))

df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Quartier', 'Status', 'Bedrooms', 'Bathrooms', 'Rooms', 'Piscine', 'Cuisine_equiped'])['Price']
    .transform(lambda x: x.mean()))

df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Quartier', 'Status', 'Bedrooms', 'Bathrooms', 'Piscine'])['Price']
    .transform(lambda x: x.mean()))

df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Quartier', 'Status', 'Bedrooms', 'Bathrooms'])['Price']
    .transform(lambda x: x.mean()))

df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Quartier', 'Status', 'Bathrooms'])['Price']
    .transform(lambda x: x.mean()))

df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Quartier', 'Status', 'Bedrooms'])['Price']
    .transform(lambda x: x.mean()))

df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Quartier', 'Status'])['Price']
    .transform(lambda x: x.mean()))

df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Quartier'])['Price']
    .transform(lambda x: x.mean()))

df['Price'] = df['Price'].fillna(
    df.groupby(['Type', 'Status'])['Price']
    .transform(lambda x: x.mean()))

In [8]:
# Fill missing Area values
df['Area'] = df['Area'].fillna(
    df.groupby(['Type', 'Quartier', 'Status', 'Bedrooms', 'Bathrooms'])['Area']
    .transform(lambda x: x.mean()))

df['Area'] = df['Area'].fillna(
    df.groupby(['Type'])['Area']
    .transform(lambda x: x.mean()))

In [10]:
def fill_property_state(row):
        if pd.isnull(row['Property_State']):
            if row['Status'] == 'Nouveau':
                return '0'
            else:
                mode_value = df[(df['Type'] == row['Type']) &
                                (df['Quartier'] == row['Quartier']) &
                                (df['Status'] == row['Status'])]['Property_State'].mode()
                if not mode_value.empty:
                    return mode_value[0]
                else:
                    mode_value = df[(df['Status'] == row['Status'])]['Property_State'].mode()
                    return mode_value[0]
        else:
            return row['Property_State']

df['Property_State'] = df.apply(fill_property_state, axis=1)

In [11]:
print(df.isnull().sum())

Type               0
Quartier           0
Price              0
Area               0
Rooms              0
Bedrooms           0
Bathrooms          0
Status             0
Property_State     0
Jardin             0
Piscine            0
Cuisine_equiped    0
dtype: int64


### Add Price_m2 column

In [13]:
# Add Price_m2 column
df['Price_m2'] = df['Price'] / df['Area']

In [14]:
print(df.head())

          Type   Quartier     Price  Area  Rooms  Bedrooms  Bathrooms  \
0  Appartement  Hay Houda  966000.0  84.0    4.0         3          2   
1  Appartement  Hay Houda  549000.0  58.0    3.0         2          2   
2  Appartement  Hay Houda  624000.0  65.0    3.0         2          2   
3  Appartement  Hay Houda  979000.0  89.0    4.0         3          2   
4  Appartement  Hay Houda  957000.0  87.0    4.0         3          2   

    Status Property_State  Jardin  Piscine  Cuisine_equiped      Price_m2  
0  Nouveau              0       0        1                1  11500.000000  
1  Nouveau              0       0        0                1   9465.517241  
2  Nouveau              0       0        0                1   9600.000000  
3  Nouveau              0       0        1                1  11000.000000  
4  Nouveau              0       0        1                1  11000.000000  


In [15]:
# Save df to csv file for further analysis and preprocessing
df.to_csv('mubawab_listings_clean.csv', index=False)